## Importing Libraries

In [1]:
import numpy as np
import keras
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import pydot
import cv2
from data_loader import load_dataset
from models import load_trial_model

### Load Dataset

In [2]:
dataset = load_dataset()

In [3]:
dataset

,image,emotion
0,facial-expressions_2868585k.jpg,surprise
1,facial-expressions_2868584k.jpg,disgust
2,facial-expressions_2868582k.jpg,fear
3,Aaron_Eckhart_0001.jpg,neutral
4,Aaron_Guiel_0001.jpg,happiness
...,...,...
13684,SharmilaTagore_80.jpg,HAPPINESS
13685,SharmilaTagore_81.jpg,HAPPINESS
13686,SharmilaTagore_82.jpg,HAPPINESS
13687,SharmilaTagore_83.jpg,HAPPINESS


In [4]:
dataset["emotion"].value_counts()

neutral      6717
happiness    5309
HAPPINESS     387
surprise      356
anger         227
DISGUST       195
NEUTRAL       151
SADNESS       144
sadness       124
ANGER          24
disgust        13
fear           13
SURPRISE       12
contempt        9
FEAR            8
Name: emotion, dtype: int64

In [5]:
#Replacing emotion in capital to small
dataset['emotion'] = dataset['emotion'].replace({'NEUTRAL':'neutral','HAPPINESS':'happiness','DISGUST':'disgust',
                                                 'SADNESS':'sadness','ANGER':'anger','SURPRISE':'surprise',
                                                 'FEAR':'fear'})

num_outputs = len(dataset["emotion"].value_counts()) #number of classes
dataset["emotion"].value_counts()


neutral      6868
happiness    5696
surprise      368
sadness       268
anger         251
disgust       208
fear           21
contempt        9
Name: emotion, dtype: int64

### Splitting into train test data

In [6]:
x_train, x_test, y_train, y_test = train_test_split(dataset['image'], dataset['emotion'])

train_data = pd.concat([x_train, y_train],axis = 1)  #training dataframe
test_data = pd.concat([x_test, y_test],axis = 1)  #test dataframe


### Using ImageDataGenerator 

In [7]:
datagen = ImageDataGenerator(rescale=1./255, validation_split = 0.2) # rescaling the pixel values between 0 to 1.
test_datagen = ImageDataGenerator(rescale=1./255) # rescaling the pixel values between 0 to 1.

#Access training images from directory marked to their class labels in dataframe
train_generator = datagen.flow_from_dataframe(dataframe = train_data,
                                              directory= "./facial_expressions/images",
                                              x_col="image", y_col = "emotion",
                                              target_size = (350,350),
                                              subset = "training")  

#Access validation images from directory marked to their class labels in dataframe
validation_generator = datagen.flow_from_dataframe(dataframe = train_data,
                                                   directory= "./facial_expressions/images",
                                                   x_col="image", y_col = "emotion",
                                                   target_size = (350,350),
                                                   subset= "validation")

#Access test images from directory marked to their class labels in dataframe
test_generator = test_datagen.flow_from_dataframe(dataframe = test_data,
                                                  directory= "./facial_expressions/images",
                                                  x_col="image", y_col = "emotion",
                                                  target_size = (350,350))

#traget size is chosen (350,350) because maximum images have this dimension.

Found 8213 validated image filenames belonging to 8 classes.
Found 2053 validated image filenames belonging to 8 classes.
Found 3423 validated image filenames belonging to 8 classes.


In [8]:
label_indices = train_generator.class_indices #get indices of each class in one hot encoding done during flow from dataframe
label_indices

{'anger': 0,
 'contempt': 1,
 'disgust': 2,
 'fear': 3,
 'happiness': 4,
 'neutral': 5,
 'sadness': 6,
 'surprise': 7}

## Load the trial model

In [9]:
model = load_trial_model((350,350,3), num_outputs)

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 173, 173, 32)      2432      
_________________________________________________________________
batch_normalization (BatchNo (None, 173, 173, 32)      128       
_________________________________________________________________
activation (Activation)      (None, 173, 173, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 86, 86, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 41, 41, 64)        51264     
_________________________________________________________________
batch_normalization_1 (Batch (None, 41, 41, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 41, 41, 64)        0

In [12]:
# Train the model
model.fit(train_generator,
          steps_per_epoch=100,
          epochs = 20,
          validation_data = validation_generator
         )


Epoch 1/20
100/100 [==============================] - 29s 294ms/step - loss: 4.0940 - accuracy: 0.5903 - val_loss: 2.0174 - val_accuracy: 0.4759
Epoch 2/20
100/100 [==============================] - 32s 318ms/step - loss: 1.2493 - accuracy: 0.6644 - val_loss: 2.7186 - val_accuracy: 0.4384
Epoch 3/20
100/100 [==============================] - 33s 329ms/step - loss: 1.0965 - accuracy: 0.6883 - val_loss: 0.7652 - val_accuracy: 0.7350
Epoch 4/20
100/100 [==============================] - 29s 290ms/step - loss: 0.7875 - accuracy: 0.7291 - val_loss: 0.7741 - val_accuracy: 0.7613
Epoch 5/20
100/100 [==============================] - 29s 291ms/step - loss: 0.6569 - accuracy: 0.7681 - val_loss: 0.8314 - val_accuracy: 0.6985
Epoch 6/20
100/100 [==============================] - 33s 331ms/step - loss: 0.6351 - accuracy: 0.7756 - val_loss: 0.7930 - val_accuracy: 0.7038
Epoch 7/20
100/100 [==============================] - 30s 298ms/step - loss: 0.6442 - accuracy: 0.7745 - val_loss: 0.7665 - val_ac

In [13]:
pred = model.evaluate(test_generator)
print("Accuracy on test set:", pred[1])
#print(np.argmax(model.predict(test_generator),axis =1))

107/107 [==============================] - 13s 122ms/step - loss: 0.6344 - accuracy: 0.7894
Accuracy on test set: 0.7893660664558411


In [17]:
model.save_weights("trial_model.h5")